In [172]:
import pickle
import itertools
import open3d as o3d
import numpy as np
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate
from scipy import ndimage
import sklearn
from sklearn.decomposition import PCA
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist


In [353]:
def edges_to_lineset(mesh, edges, color):
    ls = o3d.geometry.LineSet()
    ls.points = mesh.vertices
    ls.lines = edges
    colors = np.empty((np.asarray(edges).shape[0], 3))
    colors[:] = color
    ls.colors = o3d.utility.Vector3dVector(colors)
    return ls

def rotation_mat(angle):
    Rx = np.array([[1,0,0],
                 [0, np.cos(angle[0]), -np.sin(angle[0])],
                 [0, np.sin(angle[0]), np.cos(angle[0])]])

    Ry = np.array([[np.cos(angle[1]), 0, np.sin(angle[1])],
                 [0,1,0],
                 [-np.sin(angle[1]), 0, np.cos(angle[1])]])

    Rz = np.array([[np.cos(angle[2]), -np.sin(angle[2]), 0],
                 [np.sin(angle[2]), np.cos(angle[2]), 0],
                 [0,0,1]])
    return Rz@Rx@Ry
angle = [np.pi, np.pi, 0] # pcd.rotate()
#angle = [-np.pi/2, np.pi/2, np.pi/2]
R = rotation_mat(angle)
R_mirror = np.array([[-1,0,0],
                    [0,1,0],
                    [0,0,1]])

def get_com(mesh, edges):
    length = 500
    vertices = np.asarray(mesh.vertices)
    edges = np.asarray(edges)

    for i in range(len(vertices[edges])):
        start = vertices[edges][i][0]/100
        end = vertices[edges][i][1]/100

        _com = end + np.repeat(np.arange(length+1),3).reshape((length+1,3)) * (start - end)/length

        if i == 0:
            com = np.copy(_com)
            continue
        com = np.concatenate([com, _com])
    return com



In [354]:
curve_list=["curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_one_hole_leaf.curves",
           "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_two_hole_leaf.curves",
           "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_three_hole_leaf.curves",
           "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_four_hole_leaf.curves",
           "curves_by/fixed/smoothed/s_112_b20_s105_lmin40_lmax_100_o15_five_hole_leaf.curves",
           "curves_by/fixed/smoothed/s_112_b20_s100_lmin40_lmax_100_o15_six_hole_leaf.curves"]

In [355]:
with open(curve_list[0], "rb") as pickle_file:
    s_curve_list = pickle.load(pickle_file)

In [356]:
mesh = o3d.io.read_triangle_mesh("polygon/fix/one_hole_leaf.ply")
mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles))
edges = mesh.get_non_manifold_edges(allow_boundary_edges=False)
com = get_com(mesh, edges)
rng = np.random.default_rng()
rng.shuffle(com, axis=0)


In [357]:
x1 = np.concatenate([i.T for i in s_curve_list])
rng = np.random.default_rng()
rng.shuffle(x1, axis=0)
x1 = x1[:1500]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x1)
pcd.rotate(np.linalg.inv(R),(0,0,0))
pcd.scale(10,(0,0,0))
x1 = np.asarray(pcd.points)

In [358]:
pcd_ = o3d.geometry.PointCloud()
pcd_.points = o3d.utility.Vector3dVector(com)

In [359]:
o3d.visualization.draw_geometries([pcd,pcd_])

In [348]:
x1.shape

(1500, 3)

In [349]:
com.shape

(37575, 3)

In [350]:
x2 = com
d = cdist(x1, x2)
assignment = linear_sum_assignment(d)
emd = d[assignment].sum() / x1.shape[0]

In [351]:
emd

0.01322443046679697